# Updating Site FeatureLayer

In [ ]:
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection
import ipywidgets as widgets
import pandas as pd
import io
import warnings

def update_layer(b):    
    with output:
        # Clear any previous output
        output.clear_output()
        
        # Initialize progress bar
        progress = widgets.IntProgress(value=0, min=0, max=2, step=1, description='Progress:')
        display(progress)
        
        # Check if file is uploaded
        if uploader.value:
            
            try:
                # Read in uploaded file as a csv
                uploaded_file = next(iter(uploader.value.values()))
                csv = io.BytesIO(uploaded_file['content'])

                # Read uploaded file into a Pandas DataFrame
                df = pd.read_csv(csv)

                # Rename lat/lon
                df.rename(columns={"LongLat (Latitude)": "Longitude", "LongLat (Longitude)": "Latitude"}, inplace=True)

                # Drop null values (+ last 5 Salesforce lines)
                df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

                # Upload the csv to files
                df.to_csv('sites.csv', index=False)
                
                # Update progress
                progress.value = 1

                # Replace feature layer item
                feature_layer_item = gis.content.search(query='title:Sites', item_type='Feature Service')[0]
                collection = FeatureLayerCollection.fromitem(feature_layer_item)
                csv_path = '/arcgis/sites.csv'
                collection.manager.overwrite(csv_path)
                
                # Clean up temporary file
                os.remove('/arcgis/sites.csv')
                
                # Update progress
                progress.value = 2

                print('Sites layer updated.')
            
            except Exception as e:
                print(f"Error: {str(e)}")
                        
        else:
            print("Error: No file uploaded.")
            
# Suppress admin warning
warnings.filterwarnings("ignore", message="You are logged on as .* with an administrator role")
            
# GIS connection
gis = GIS("home")            
            
# Initialize widgets and button click event
output = widgets.Output()
uploader = widgets.FileUpload(accept='.csv', multiple=False)
button = widgets.Button(description='Update Layer', tooltip='Click me')
button.on_click(update_layer)

# Display widgets and button
print('\nUpload LimnoTech Sites report (csv format).')
display(uploader)

print('\nClick the button to update the feature layer.')
display(button)
display(output)